In [16]:
%load_ext autoreload
%autoreload 2
import torch
import torch.nn.functional as F
from torch import Tensor
from datetime import datetime
import math
import random
import numpy as NP
import matplotlib.pyplot as plt
%matplotlib inline
relPath = "../../"
import sys
sys.path.append(relPath)
from mm.printing import *
from mm.neural.neural import *
from introspection import *
from layers import *
from network import *
from neural import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
initLogging("Common init")

dtype = torch.float
log("dtype", dtype)

dvc = torch.device("cpu")
log("device", dvc.type)

contextSize = 3
log("contextSize", contextSize)

newNet = True
log("newNet", newNet)


Common init:         -------------------------- 2023-07-21 14:44:47
dtype:               torch.float32
device:              cpu
contextSize:         3
newNet:              True


In [18]:
logSection("Prepare data")

filePath = relPath + "data/names.txt"
log("filePath", filePath)

trRatio = 0.8
log("trRatio", trRatio)

devRatio = 0.9
log("devRatio", devRatio)

wordShufflingSeed = 42
log("wordShufflingSeed", wordShufflingSeed)

words = readFileSplitByLine(filePath)
random.seed(wordShufflingSeed)
random.shuffle(words)
log("first few words", words[:5])

lenWords = len(words);
log("lenWords", lenWords)

allPossibleChars = sorted(list(set("".join(words))))
log("allPossibleChars", allPossibleChars)

stoi = sToI(allPossibleChars)
log("stoi", stoi)

itos = iToS(stoi)
log("itos", itos)

vocabularyLength = len(itos)
log("vocabularyLength", vocabularyLength)

log("data random probability", f"{-torch.tensor(1 / vocabularyLength).log().item():.4f}")


Prepare data:        -------------------------- 2023-07-21 14:44:48
filePath:            ../../data/names.txt
trRatio:             0.8
devRatio:            0.9
wordShufflingSeed:   42
first few words:     ['yuheng', 'diondre', 'xavien', 'jori', 'juanluis']
lenWords:            32033
allPossibleChars:    ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
stoi:                {'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '.': 0}
itos:                {1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
vocabularyLength:    27
data random probability: 3.295

In [19]:
logSection("Prepare dataset")

dataDtype = torch.int64
log("data dtype", dataDtype)

lenTrain = int(trRatio * lenWords)
trWords = words[:lenTrain]
trX, trY = buildDataSet(trWords, contextSize, stoi, itos, dataDtype, dvc)
log("data set training", lenTrain, trX.shape, trY.shape, trWords[:3])

endVal = int(devRatio * lenWords)
valWords = words[lenTrain:endVal];
valX, valY = buildDataSet(valWords, contextSize, stoi, itos, dataDtype, dvc)
log("data set validation", endVal - lenTrain, valX.shape, valY.shape, valWords[:3])

lenTest = lenWords - endVal
tstWords = words[endVal:]
tstX, tstY = buildDataSet(tstWords, contextSize, stoi, itos, dataDtype, dvc)
log("data set test", lenTest, tstX.shape, tstY.shape, tstWords[:3])

Prepare dataset:     -------------------------- 2023-07-21 14:44:48
data dtype:          torch.int64
data set training:   25626 torch.Size([182625, 3]) torch.Size([182625]) ['yuheng', 'diondre', 'xavien']
data set validation: 3203 torch.Size([22655, 3]) torch.Size([22655]) ['amay', 'aytana', 'jenevi']
data set test:       3204 torch.Size([22866, 3]) torch.Size([22866]) ['mustafa', 'reuben', 'kahlel']


In [20]:
logSection("Build network")

embeddingDims = 10
log("embeddingDims", embeddingDims)

hiddenLayerSize = 200
log("hiddenLayerSize", hiddenLayerSize)

learningSeed = 2147483647
log("learningSeed", learningSeed)
g = torch.Generator(device=dvc).manual_seed(learningSeed)

np2 = makeNetwork2(g, vocabularyLength, embeddingDims, contextSize, hiddenLayerSize, dtype, dvc)
printNetwork(np2)
    
@torch.no_grad()
def trLoss2(): return getLoss2(np2, np2.C[trX], trY)

@torch.no_grad()
def valLoss2(): return getLoss2(np2, np2.C[valX], valY)

@torch.no_grad()
def tstLoss2(): return getLoss2(np2, np2.C[tstX], tstY)

def getLosses2() -> Losses2:
    l = Losses2()
    l.tr = trLoss2()
    l.val = valLoss2()
    l.tst = tstLoss2()
    return l

def logLosses2():
    losses = getLosses2()
    l1 = f"{losses.tr.loss.item():>10.4f}"
    l2 = f"{losses.val.loss.item():>10.4f}"
    l3 = f"{losses.tst.loss.item():>10.4f}"
    logSimple(f"{l1} {l2} {l3}")

Build network:       -------------------------- 2023-07-21 14:44:48
embeddingDims:       10
hiddenLayerSize:     200
learningSeed:        2147483647
Network Structure:  
Layer LinearWithBias 22284: torch.Size([30, 200]), torch.Size([200]), 
Layer Tanh 22285: 
Layer LinearWithBias 22286: torch.Size([200, 200]), torch.Size([200]), 
Parameters Count:    46670


In [21]:
logSection("Learning")

trainingBatchSize = 32
log("trainingBatchSize", trainingBatchSize)

trXLength = trX.shape[0]
trainingSteps = (trXLength // trainingBatchSize) + 1
log ("trainingSteps: ", trainingSteps)

maxIteration = 20_000
log("maxIteration", maxIteration)

repeats = 36
log("repeats",repeats)

maxLr = 0.14
log("maxLr", maxLr)

minLr = 0.0001
log("minLr", minLr)

actualIterations = min(maxIteration, repeats * math.ceil(trXLength / trainingBatchSize))
log("actualIterations", actualIterations)

lrAtIx: list[float] = []
stepIx: list[int] = []
lossAtIx: list[float] = []
logLossAtIx: list[float] = []
up = UpdateNetResult()
fr = ForwardPassResult2()
fr.loss = torch.tensor(0)
i = 0
bnMeanRunning = 0
bnStdRunning = 0
for repeat in range(repeats):
    
    if i >= maxIteration:
        break;

    logSimple(f"{repeat:>3}, {i:>6} losses: {fr.loss.item():>10.4f}   ", end="")
    logLosses2()

    for start in range(0, trXLength, trainingBatchSize):

        if i >= maxIteration:
            log("Break at max iteration")
            break;
        
        end = min(start + trainingBatchSize, trXLength)
        miniBatchIxs = torch.randint(0, trXLength, (trainingBatchSize,), generator=g, device=dvc)
        fr = forwardPass2(np2, trX, trY, miniBatchIxs)
        backwardPass2(np2.layers, np2.parameters, fr.loss)

        up = updateNet(np2.parameters, i, actualIterations, maxLr, minLr)
        lrAtIx.append(up.learningRate)

        stepIx.append(i)
        lossAtIx.append(fr.loss.item())
        logLossAtIx.append(fr.loss.log10().item())

        i += 1

logSimple(f" final losses: {fr.loss.item():>15.4f}   ", end="")
logLosses2()

#bestLr = lrs[findLowestIndex(lossAtIx)].item();
#log("best learning rate", bestLr)
log("emb.shape", fr.emb.shape)
#log("h.shape", fr.h.shape)
log("logits.shape", fr.logits.shape)

Learning:            -------------------------- 2023-07-21 14:44:48
trainingBatchSize:   32
trainingSteps: :     5708
maxIteration:        20000
repeats:             36
maxLr:               0.14
minLr:               0.0001
actualIterations:    20000
  0,      0 losses:     0.0000       5.2987     5.2984     5.2988
  1,   5708 losses:     2.7872       2.3467     2.3427     2.3464
  2,  11416 losses:     2.4146       2.3017     2.3005     2.3052
  3,  17124 losses:     2.0725       2.2665     2.2678     2.2699
Break at max iteration:
 final losses:          1.9495       2.2575     2.2586     2.2608
emb.shape:           torch.Size([32, 3, 10])
logits.shape:        torch.Size([32, 200])


In [22]:
logSection("Sampling")

samplingSeed = learningSeed + 10
gSampling = torch.Generator(device=dvc).manual_seed(samplingSeed)
log("samplingSeed", samplingSeed)

maxSampleLength = 50
log("maxSampleLength", maxSampleLength)

Sampling:            -------------------------- 2023-07-21 14:45:10
samplingSeed:        2147483657
maxSampleLength:     50


In [50]:
samples = sampleMany2(np2, gSampling, contextSize, itos, 20, maxSampleLength)
for s in samples:
    logSimple(f"{''.join(s.values):<21}{(s.prob * 10000):>4.0f}: ", end="")
    for p in s.probs:
        logSimple(f"{(p / (1 / 27) * 10):.0f} ", end="")
    logSimple()

KeyError: 190

In [36]:
if newNet:
    pass
else:
    def printProb(txt: str):
        ps = calcProb(np, cal, txt, contextSize, stoi)
        op = calcOneProb(ps)
        logSimple(f"{txt:<21}{(op * 10000):<7.0f}: ", end="")
        for p in ps:
            logSimple(f"{(p / (1 / 27) * 10):.0f} ", end="")
        logSimple()

    printProb('.')
    printProb('m.')
    printProb('mi.')
    printProb('mic.')
    printProb('mich.')
    printProb('micha.')
    printProb('michal.')
    printProb('michael.')
    printProb('michaela.')
    printProb('michaella.')
    printProb('michel.')
    printProb('michalx.')
    printProb('michalxx.')
    printProb('michalxxx.')
    printProb('martin.')
    printProb('andrej.')
    printProb('andrey.')
    printProb('joey.')
    printProb('james.')
    printProb('xin.')
    printProb('maxim.')
    printProb('alex.')
    printProb('alexa.')

In [28]:
if newNet:
    pass
else:
    plt.plot(range(len(lrAtIx)), lrAtIx, "black")
    plt.ylim(min(lrAtIx), max(lrAtIx))
    plt.grid(True)
    plt.show()
    print("Actual min max LR", max(lrAtIx), min(lrAtIx))

In [29]:
if newNet:
    pass
else:
    plt.figure(figsize=(20,10))
    plt.imshow(fr.h.abs() > 0.99, cmap="gray", interpolation="nearest")

In [30]:
if newNet:
    pass
else:
    fig, ax = plt.subplots()
    fig.set_facecolor("#777777")
    ax.set_facecolor("#222222")
    #ax.plot(lrAtIx, lossAtIx)
    plt.show()

In [31]:
if newNet:
    pass
else:
    fig, ax = plt.subplots()
    fig.set_facecolor("#777777")
    ax.set_facecolor("#222222")
    ax.plot(stepIx, lossAtIx)
    plt.show()

In [32]:
if newNet:
    pass
else:
    fig, ax = plt.subplots()
    fig.set_facecolor("#777777")
    ax.set_facecolor("#222222")
    ax.plot(stepIx, logLossAtIx)
    plt.show()

In [33]:
if newNet:
    pass
else:
    dim = 0
    fig = plt.figure(figsize=(8,8))
    fig.set_facecolor("#777777")
    sc = plt.scatter(np.C[:, dim].data, np.C[:,dim + 1].data, s=200)
    for i in range(np.C.shape[0]):
        plt.text(np.C[i, dim].item(), np.C[i, dim + 1].item(), itos[i], ha="center", va="center", color="white")
    plt.grid()

In [34]:
#np.C.shape, trX.shape, np.C[trX].shape, np.C[:5], trX[:5], np.C[trX][:5]

In [35]:
# Let"s suppose these are your lists
C = [
[ 0.8774, -0.6801],
         [ 0.1651, -0.5025],
         [ 0.2769, -0.3570],
         [-0.8820,  0.3902],
         [-0.4824,  0.8744],
         [-0.3190,  0.7807],
         [-0.0100, -0.3401],
         [ 0.9975,  2.8280],
         [ 0.9623, -1.3172],
         [ 0.2180, -0.3820],
         [ 0.6139, -0.4287],
         [-0.7386,  0.5880],
         [-0.3088,  0.9816],
         [ 0.3907, -0.4174],
         [-0.7380,  0.5205],
         [-0.5288,  0.7074],
         [-0.3956,  0.9625],
         [-0.3802, -0.3504],
         [-0.2861,  0.7589],
         [ 0.5309, -0.5105],
         [-0.0922, -0.6410],
         [-0.3823,  0.9899],
         [ 0.0965, -0.5708],
         [-0.8582, -1.3429],
         [-0.4960,  0.2842],
         [-0.6105,  0.1336],
         [-0.2623,  0.2942]
]
trX = [
    [0, 0, 0],
    [0, 0, 22],
    [0, 22, 9],
    [22, 9, 15],
    [9, 15, 12],
    # ... More values
]

# We create a new list to hold the result
result = []

# We loop over each element in trX
for i in range(len(trX)):
    temp = []
    # Then we loop over each value in the element
    for j in range(len(trX[i])):
        # We use the value as an index to get the corresponding data from C
        temp.append(C[trX[i][j]])
    # We add the result to our new list
    result.append(temp)

# Let"s log the first 5 elements of the result list
for i in range(5):
    print(result[i])

[[0.8774, -0.6801], [0.8774, -0.6801], [0.8774, -0.6801]]
[[0.8774, -0.6801], [0.8774, -0.6801], [0.0965, -0.5708]]
[[0.8774, -0.6801], [0.0965, -0.5708], [0.218, -0.382]]
[[0.0965, -0.5708], [0.218, -0.382], [-0.5288, 0.7074]]
[[0.218, -0.382], [-0.5288, 0.7074], [-0.3088, 0.9816]]
